In [ ]:
!python3 -m pip install --upgrade pip
!apt-get update
!apt-get upgrade -y
!pip install pretty_midi
!pip install pandas
!pip install matplotlib

     |████████████████████████████████| 1.7 MB 13.7 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:11 http://ppa.launchpad.

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pretty_midi
import os
import csv

In [ ]:
embedding_dim = 512
start_key = 21
history = 480 # 960 # 480 # 1920
epochs = 30
batch_size = 128
fs = 8 # 16 # 4  # 32
temperature = 1.0

In [ ]:
def get_file_paths(path):
    midi_files = []
    for root_dir, sub_dir, files in os.walk(path):
        for name in files:
            if name[0] != '.':
                midi_files.append(os.path.join(root_dir, name))
    # print(midi_files)
    return midi_files

In [ ]:
midi_files = get_file_paths('/content/drive/MyDrive/Colab Notebooks'+'/Schumann/')

In [ ]:
print(midi_files)
print(len(midi_files))

['/content/drive/MyDrive/Colab Notebooks/Schumann/sm_etude.mid', '/content/drive/MyDrive/Colab Notebooks/Schumann/varsif01.mid', '/content/drive/MyDrive/Colab Notebooks/Schumann/varsif03.mid', '/content/drive/MyDrive/Colab Notebooks/Schumann/varsif02.mid', '/content/drive/MyDrive/Colab Notebooks/Schumann/sc_ps141.mid', '/content/drive/MyDrive/Colab Notebooks/Schumann/sr12-3.mid', '/content/drive/MyDrive/Colab Notebooks/Schumann/schm17_1.mid', '/content/drive/MyDrive/Colab Notebooks/Schumann/schnson1.mid', '/content/drive/MyDrive/Colab Notebooks/Schumann/sr12-4.mid', '/content/drive/MyDrive/Colab Notebooks/Schumann/schnson2.mid', '/content/drive/MyDrive/Colab Notebooks/Schumann/sr12-5.mid', '/content/drive/MyDrive/Colab Notebooks/Schumann/schnson3.mid', '/content/drive/MyDrive/Colab Notebooks/Schumann/schm1202.mid', '/content/drive/MyDrive/Colab Notebooks/Schumann/sr12-7.mid', '/content/drive/MyDrive/Colab Notebooks/Schumann/schnson4.mid']
15


In [ ]:
def prettify(midi_files):
    pretty_files = []
    for f in midi_files:
        try:
            pretty_files.append(pretty_midi.PrettyMIDI(f))
        except:
            print('Err File: ' + f)
    return pretty_files

In [ ]:
prettified = prettify(midi_files)

In [ ]:
# tmp_fs = fs
# fs=100
def notes_to_dict(pretty_files, fs):
    musics = []
    for i in range(len(pretty_files)):
        music = dict()
        lines = dict()
        instruments = pretty_files[i].instruments
        # Merge instruments
        tracks = pd.DataFrame([])
        for inst in instruments:
            track = inst.get_piano_roll(fs=fs)
            track_df = pd.DataFrame(data=track)
            tracks = pd.concat([tracks, track_df]).groupby(level=0).max()
        tracks = tracks.values
        for key_index in range(tracks.shape[0]):
            duration = tracks[key_index] # Numeric
            # duration = np.array([str(int(x)) for x in duration]) # Str
            lines[key_index] = duration
        music = dict(list(music.items()) + list(lines.items()))
        musics.append(music)
    return musics
# fs = tmp_fs

In [ ]:
notes_dict = notes_to_dict(prettified, fs)

In [ ]:
# Data Preparation

dataset = np.array([])
for nd in notes_dict:
    durations = np.array(list(nd.values())).reshape(128, -1)
    # tmp_dur = np.transpose(durations)
    '''
    for i in range(len(tmp_dur)):
        if any(v != 0 for v in tmp_dur[i]):
            tmp_dur = tmp_dur[i:]
            break
    for i in range(len(tmp_dur)):
        if any(v != 0 for v in tmp_dur[-i]):
            tmp_dur = tmp_dur[-i:]
            break
    '''
    # durations = np.transpose(tmp_dur)
    if len(dataset) == 0:
        dataset = durations
    else:
        dataset = np.hstack((dataset, durations))
        print(dataset.shape)
dataset = dataset[start_key: start_key + 88]
print(dataset.shape)

(128, 8533)
(128, 13024)
(128, 15657)
(128, 19753)
(128, 20537)
(128, 25685)
(128, 28560)
(128, 30384)
(128, 32189)
(128, 34317)
(128, 35083)
(128, 36339)
(128, 37651)
(128, 40135)
(88, 40135)


In [ ]:
# Convert data to binary string representation

binary_dataset = np.sign(dataset)
uniq_data = np.unique(binary_dataset, axis=1)
print('data shape = ' + str(dataset.shape))
print('uniq data shape = ' + str(uniq_data.shape))
uniq_data_transpose = np.transpose(uniq_data)

uniq_data_str = []
for i in uniq_data:
    tmp = [str(int(j)) for j in i]
    uniq_data_str.append(tmp)
uniq_data_str = np.array(uniq_data_str)
print('uniq data string shape = ' + str(uniq_data_str.shape))

binary_dataset_transpose = np.transpose(binary_dataset)
binary_data_str = []
for i in binary_dataset_transpose:
    tmp = [str(int(j)) for j in i]
    binary_data_str.append(''.join(tmp))
binary_data_str = np.array(binary_data_str)
print('binary data str shape = ' + str(binary_data_str.shape))

data shape = (88, 40135)
uniq data shape = (88, 10573)
uniq data string shape = (88, 10573)
binary data str shape = (40135,)


In [ ]:
uniq_data_str_transpose = np.transpose(uniq_data_str)
id2str = {}
for i in range(len(uniq_data_str_transpose)):
    id2str.update({i: ''.join(uniq_data_str_transpose[i])})

str2id = {s:i for i, s in id2str.items()}

print(id2str[1])
print(str2id['0000000000000000000000000000000000000000000000000000000000000000000000000000001000000000'])

0000000000000000000000000000000000000000000000000000000000000000000000000000001000000000
1


In [ ]:
data = []
target = []
start_index = history
end_index = binary_data_str.shape[0] - 1
for i in range(start_index, end_index-1):
    x_indicies = range(i-history, i)
    y_indicies = range(i-history+1, i+1)
    # data.append(np.reshape(dataset[indicies],
    #                        (history, self.embedding_dim)))
    # target.append(dataset[i+1])
    # tmp_data = [','.join(x) for x in dataset[indicies]]
    # data.append(tmp_data)
    # target.append(np.reshape(','.join(dataset[i+1]), 1))
    # data.append(np.reshape([str2id[x] for x in dataset[indicies]],
    #                        (history, 1)))
    '''
    data.append(np.reshape([str2id[x] for x in dataset[x_indicies]],
                                   (history, 1)))
    target.append(np.reshape([str2id[x] for x in dataset[y_indicies]],
                                     (history, 1)))
    '''
    data.append([str2id[x] for x in binary_data_str[i-history:i]])
    target.append([str2id[binary_data_str[i+1]]])
data = np.array(data)
target = np.array(target)

target = np.squeeze(target)
target = tf.keras.utils.to_categorical(target, num_classes=len(str2id))

print(data.shape)
print(target.shape)

(39653, 480)
(39653, 10573)


In [ ]:
data[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,  8874,
        8874,  8874,  8874,  8874,  8874,  8874,  8874,  8874,  8874,
        8882,  8882,  8882,  8882,  8882,  8882,  8882,  8882,  8882,
        8890,  8890,  8890,  8890,  8890,  8890,  8890,  8890,  8890,
        8890,  8926,  8926,  8926,  8926,  8926,  8926,  8926,  8926,
       10284, 10330, 10331, 10331, 10331, 10331, 10331, 10331, 10331,
       10331, 10331, 10331, 10333, 10333, 10333, 10333, 10333, 10334,
       10334,  3020, 10543, 10544, 10545, 10545, 10545, 10545, 10545,
       10545, 10545, 10545, 10545, 10545, 10545, 10545, 10545, 10545,
       10545, 10545, 10545, 10545, 10545,   329,  9943, 10020, 10026,
       10026, 10026, 10026, 10026, 10026, 10026, 10026, 10026, 10026,
       10027, 10027, 10027, 10027, 10027, 10027, 10027, 10027, 10527,
       10549, 10552, 10552, 10552, 10552, 10552, 10552, 10552, 10552,
       10284, 10353, 10358, 10358, 10358, 10358, 10358, 10358, 10358,
       10358,  9943,

In [ ]:
target[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [ ]:
class TransformerEncoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super(TransformerEncoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = tf.keras.Sequential(
            [tf.keras.layers.Dense(dense_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),]
        )
        self.layernorm_1 = tf.keras.layers.LayerNormalization()
        self.layernorm_2 = tf.keras.layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'dense_dim': self.dense_dim,
            'num_heads': self.num_heads,
            'embed_dim': self.embed_dim
        })
        return config

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super(PositionalEmbedding, self).__init__(**kwargs)
        self.token_embeddings = tf.keras.layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = tf.keras.layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'sequence_length': self.sequence_length,
            'vocab_size': self.vocab_size,
            'embed_dim': self.embed_dim
        })
        return config

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(tf.keras.layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super(TransformerDecoder, self).__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = tf.keras.Sequential(
            [tf.keras.layers.Dense(latent_dim, activation="relu"), tf.keras.layers.Dense(embed_dim),]
        )
        self.layernorm_1 = tf.keras.layers.LayerNormalization()
        self.layernorm_2 = tf.keras.layers.LayerNormalization()
        self.layernorm_3 = tf.keras.layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'num_heads': self.num_heads,
            'latent_dim': self.latent_dim,
            'embed_dim': self.embed_dim
        })
        return config

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [ ]:
embed_dim = 128
latent_dim = 512
num_heads = 8
ff_dim = 128
vocab_size = len(str2id)

encoder_inputs = tf.keras.Input(shape=(history,), name="encoder_inputs")
x = PositionalEmbedding(history, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = tf.keras.Model(inputs=encoder_inputs, outputs=encoder_outputs)

decoder_inputs = tf.keras.Input(shape=(history,), name="decoder_inputs")
encoded_seq_inputs = tf.keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(history, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dropout(0.1)(x)
decoder_outputs = tf.keras.layers.Dense(vocab_size, activation="softmax")(x)
decoder = tf.keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])

model_transformer = tf.keras.Model(
    inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs, name="transformer"
)

In [ ]:
model_transformer.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, 480)]        0           []                               
                                                                                                  
 positional_embedding (Position  (None, 480, 128)    1414784     ['encoder_inputs[0][0]']         
 alEmbedding)                                                                                     
                                                                                                  
 decoder_inputs (InputLayer)    [(None, 480)]        0           []                               
                                                                                                  
 transformer_encoder (Transform  (None, 480, 128)    659712      ['positional_embedding[

In [ ]:
print(data.shape)
target = data[1:]
data = data[:-1]
print(data.shape)
print(target.shape)

(39653, 480)
(39652, 480)
(39652, 480)


In [ ]:
data[0]

array([    0,     0,     0,     0,     0,     0,     0,     0,  8874,
        8874,  8874,  8874,  8874,  8874,  8874,  8874,  8874,  8874,
        8882,  8882,  8882,  8882,  8882,  8882,  8882,  8882,  8882,
        8890,  8890,  8890,  8890,  8890,  8890,  8890,  8890,  8890,
        8890,  8926,  8926,  8926,  8926,  8926,  8926,  8926,  8926,
       10284, 10330, 10331, 10331, 10331, 10331, 10331, 10331, 10331,
       10331, 10331, 10331, 10333, 10333, 10333, 10333, 10333, 10334,
       10334,  3020, 10543, 10544, 10545, 10545, 10545, 10545, 10545,
       10545, 10545, 10545, 10545, 10545, 10545, 10545, 10545, 10545,
       10545, 10545, 10545, 10545, 10545,   329,  9943, 10020, 10026,
       10026, 10026, 10026, 10026, 10026, 10026, 10026, 10026, 10026,
       10027, 10027, 10027, 10027, 10027, 10027, 10027, 10027, 10527,
       10549, 10552, 10552, 10552, 10552, 10552, 10552, 10552, 10552,
       10284, 10353, 10358, 10358, 10358, 10358, 10358, 10358, 10358,
       10358,  9943,

In [ ]:
model_transformer.compile(optimizer=tf.keras.optimizers.Adam(),
                          loss='categorical_crossentropy',
                          metrics = ["accuracy"]
                         )
model_history = model_transformer.fit(x=[data, data], y=target, epochs=epochs,
                          batch_size=batch_size,
                          shuffle=True,
                          validation_split=0.1
                         )
model_transformer.save('/content/drive/MyDrive/Colab Notebooks/Music_models' \
                    + '/schumann_transformerEncoderDecoder_binary_str_ce_' \
            + str(history) + '_ep_' + str(epochs) + '_' + str(embed_dim) + '_dim_categorica.h5')

Epoch 1/30
279/279 [==============================] - 100s 337ms/step - loss: 8.7013 - accuracy: 0.0436 - val_loss: 9.4527 - val_accuracy: 0.0416
Epoch 2/30
279/279 [==============================] - 93s 334ms/step - loss: 7.7825 - accuracy: 0.0449 - val_loss: 9.8825 - val_accuracy: 0.0270
Epoch 3/30
279/279 [==============================] - 93s 335ms/step - loss: 6.9925 - accuracy: 0.0576 - val_loss: 10.5181 - val_accuracy: 0.0174
Epoch 4/30
279/279 [==============================] - 93s 335ms/step - loss: 6.1841 - accuracy: 0.0810 - val_loss: 10.9153 - val_accuracy: 0.0035
Epoch 5/30
279/279 [==============================] - 93s 335ms/step - loss: 5.4119 - accuracy: 0.1128 - val_loss: 11.5923 - val_accuracy: 0.0023
Epoch 6/30
279/279 [==============================] - 93s 334ms/step - loss: 4.7129 - accuracy: 0.1524 - val_loss: 12.4963 - val_accuracy: 7.5643e-04
Epoch 7/30
279/279 [==============================] - 93s 334ms/step - loss: 4.0908 - accuracy: 0.1986 - val_loss: 12.907

In [ ]:
conf = model_transformer.get_config()

In [ ]:
print(conf)

{'name': 'transformer', 'layers': [{'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 480), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'encoder_inputs'}, 'name': 'encoder_inputs', 'inbound_nodes': []}, {'class_name': 'PositionalEmbedding', 'config': {'name': 'positional_embedding', 'trainable': True, 'dtype': 'float32', 'sequence_length': 480, 'vocab_size': 10573, 'embed_dim': 128}, 'name': 'positional_embedding', 'inbound_nodes': [[['encoder_inputs', 0, 0, {}]]]}, {'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 480), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'decoder_inputs'}, 'name': 'decoder_inputs', 'inbound_nodes': [], 'shared_object_id': 2}, {'class_name': 'TransformerEncoder', 'config': {'name': 'transformer_encoder', 'trainable': True, 'dtype': 'float32', 'dense_dim': 512, 'num_heads': 8, 'embed_dim': 128}, 'name': 'transformer_encoder', 'inbound_nodes': [[['positional_embedding', 0, 0, {}]]]}, {'class_

In [ ]:
# Generate

def prettify_generate(notes_generated, num_generate):
    notes_matrix = []
    notes_generated = np.squeeze(notes_generated)
    for i in range(num_generate):
        tmp = []
        for j in range(88):
            tmp.append(int(notes_generated[i][j]))
        notes_matrix.append(tmp)
        # notes_matrix.append([int(x) for x in notes_generated[i].split(',')])
    print(np.array(notes_matrix).shape)
    notes_matrix = np.transpose(notes_matrix)
    print(notes_matrix.shape)
    notes_dict = {}
    for i in range(88):
        notes_dict[i] = list(notes_matrix[i])
    print(len(notes_dict))
    notes_matrix = np.array(list(notes_dict.values()))
    name = 'schumann_transformer_encoder_decoder_fs' + str(fs) + '_hist' + str(history) +\
                    '_tp' + str(temperature) + '_emb' +\
                str(embedding_dim) + '.mid'
    music_name = '/content/drive/MyDrive/Colab Notebooks' + '/Music_generate/' + name
    piano_roll_to_pretty_midi(notes_matrix, music_name, fs)

def piano_roll_to_pretty_midi(piano_roll, file_name, fs, program=0):
    '''Convert a Piano Roll array into a PrettyMidi object with a single
    instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    # pad 1 column of zeros so we can acknowledge inital and ending events
    piano_roll = np.pad(piano_roll, [(0, 0), (1, 1)], 'constant')

    # use changes in velocities to find note on / note off events
    velocity_changes = np.nonzero(np.diff(piano_roll).T)

    # keep track on velocities and note on times
    prev_velocities = np.zeros(notes, dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        # use time + 1 because of padding above
        velocity = piano_roll[note, time + 1]
        time = time / fs
        if velocity > 0:
            if prev_velocities[note] == 0:
                note_on_time[note] = time
                prev_velocities[note] = velocity
        else:
            pm_note = pretty_midi.Note(
                                        velocity=prev_velocities[note],
                                        pitch=note,
                                        start=note_on_time[note],
                                        end=time)
            instrument.notes.append(pm_note)
            prev_velocities[note] = 0
    pm.instruments.append(instrument)
    for i in range(len(pm.instruments)):
        for note in pm.instruments[i].notes:
            note.velocity = 100
    pm.write(file_name)
    print('Music Generated!!')

In [ ]:
notes_generated = []

def generator(model, data, target, vocab_size, history, str2id, batch_size):
    # model = build_model(data, target, vocab_size, history, str2id, 1)
    custom_objects = {"TransformerEncoder": TransformerEncoder,
                      "TransformerDecoder": TransformerDecoder,
                      "PositionalEmbedding": PositionalEmbedding
                      }
    config = model.get_config()
    # with tf.keras.utils.custom_object_scope(custom_objects):
    #     new_model = tf.keras.Model.from_config(config)

    model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Music_models' \
                    + '/schumann_transformerEncoderDecoder_binary_str_ce_' \
            + str(history) + '_ep_' + str(epochs) + '_' + str(embed_dim) + '_dim_categorica.h5',
            custom_objects=custom_objects)
    # model.load_weights(tf.train.latest_checkpoint(self.path +
    #                                             '/Music_models/checkpoints/'))
    # model.build(tf.TensorShape([None, 1]))
    # model.save(self.path + '/Music_models/schumann_lstm_' + str(self.history) + '_'\
    #         + str(self.epochs) + '_embedding_categorical_gen.h5')
    # model.summary()
    print('Load model suscess!')
    num_generate = fs * 60 * 2 # 2 min
    notes_generated = []
        
    # temperature = temperature # Prediction regulator
    # Small temperature -> real pred. Large temperature -> surprizing 

    starter = data[-1]
    starter = tf.expand_dims(starter, 0)
    # print(starter[0])

    # model.reset_states()
    preds_memory = []
    for i in range(num_generate):
        pred = model([starter, starter])
        pred = tf.squeeze(pred, 0)
        pred = pred / temperature
        # print(pred)
        pred = tf.expand_dims(pred, axis=0)
        # print(pred.shape)
        # pred_id = tf.random.categorical(pred, 1)[-1, 0].numpy()
        pred_id = tf.random.categorical(pred, 1)[-1, 0].numpy()
        # print(pred_id.shape)
        preds_memory.append(pred_id)
        if len(preds_memory) >= 160:
            preds_memory.pop(0)
        while pred_id in preds_memory:
            starter = tf.expand_dims(data[-i], 0)
            pred_inps = [starter, starter]
            pred = model(pred_inps)
            pred = tf.squeeze(pred, 0)
            pred = pred / temperature
            pred = tf.expand_dims(pred, axis=0)
            pred_id = tf.random.categorical(pred, 1)[-1, 0].numpy()
        starter = tf.expand_dims([pred_id], 0)

        notes_generated.append(id2str[pred_id])
    # print(notes_generated[0])
    notes_generated = np.array(notes_generated)
    print('notes_generated shape = ' + str(notes_generated.shape))
    print(notes_generated[0])
    prettify_generate(notes_generated, num_generate)
    return notes_generated

In [ ]:
notes = generator(model_transformer, data, target, len(str2id), history, str2id, 1)

Load model suscess!
notes_generated shape = (960,)
0000000000000000000000000001000000000001000000001000000000000000000000000000000000000000
(960, 88)
(88, 960)
88
Music Generated!!
